In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import copy

In [4]:
dataset = pd.read_csv('apparel-trainval.csv')
# dataset

In [5]:
X = dataset.drop('label', axis = 1)
Y = dataset['label']

In [6]:
X = (X - np.mean(X))/np.std(X)

In [7]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2)

In [8]:
X_train = X_train.iloc[:]
# X_copy = copy.deepcopy(X_train)
# Y_copy = copy.deepcopy(Y_train)
# Y_train
Y_train=pd.concat([Y_train,pd.get_dummies(Y_train,prefix='label')],axis=1).drop(['label'],axis=1)
Y_train = Y_train.iloc[:]

In [9]:
class Neural_network(object):
    
    def __init__(self):
        self.input_layer_size = 784
        self.output_layer_size = 10
        self.hidden_layer_size = 70
        self.hidden_layers = 1
        self.weights = {}
        self.dweights = {}
        self.Z = {}
        self.a = {}
        self.Y_hat = np.array([])
        self.delta = {}
        self.epoch = 300
        self.ita = 0.001
        
    def initialization(self):
        
        self.weights[1] = np.random.randn(self.input_layer_size,self.hidden_layer_size)*0.01
        self.weights[self.hidden_layers+1] = np.random.randn(self.hidden_layer_size,self.output_layer_size)*0.01
        
        for i in np.arange(2,self.hidden_layers+1):
            self.weights[i] = np.random.randn(self.hidden_layer_size,self.hidden_layer_size)*0.01
            
    
    def softmax(self,a):
        tsum = np.sum(a,axis = 1,keepdims = True)
        soft_a = np.divide(a,tsum)
        return soft_a
    
#     def y_prep(self,Y,k):
#          # Onehot
#         y = np.zeros((len(Y),self.output_layer_size))
#         for i in range(y.shape[0]):
#             y[i][Y[i]] = 1
#         return y
    
    
    def error(self,y):
        return sum(0.5*np.sum((y-self.Y_hat)*(y-self.Y_hat),axis = 1))/y.shape[0]
    
    def derivative_sigmoid(self,layer):
        derivative = layer*(1-layer)
        return derivative
    
   
    def deltas(self,y):
        w = self.a[self.hidden_layers+2]
        self.delta[self.hidden_layers+2] = -(y - self.Y_hat)*self.derivative_sigmoid(w)
        self.dweights[self.hidden_layers+1] = np.dot(self.a[self.hidden_layers+1].T , (self.delta[self.hidden_layers+2]))
        
        for i in np.arange(self.hidden_layers+1,1,-1):
            self.delta[i] = ( np.dot(self.delta[i+1], self.weights[i].T))*self.derivative_sigmoid(self.a[i])
            self.dweights[i-1] = np.dot(self.a[i-1].T , (self.delta[i]))  
    
    def sigmoid(self,x):
        x = -x
        return 1 / (1 + np.exp(x))
    
    
    
    def forward_propagation(self, X):
        self.a[1]=np.array(X)
        

        for i in range(1,self.hidden_layers+2):
            self.Z[i+1] = np.dot(self.a[i],self.weights[i])
            self.a[i+1] = self.sigmoid(self.Z[i+1])
        self.Y_hat = self.softmax(self.a[self.hidden_layers+2])
        
   
    def gradient_descent(self,X_train,Y_train):
        k = 0
        while k<=47999:
#             X=X_train.iloc[k:k+1000]
#             Y=Y_train.iloc[k:k+1000]
            X = X_train[k:k+1000]
            y = Y_train[k:k+1000]
#             print(y)
            for i in range(self.epoch):
                self.forward_propagation(X)
#                 y = self.y_prep(Y,k)
#                 print("Error : ",self.error(y))
                self.deltas(y)
                for j in range(1,self.hidden_layers+2):
                    self.weights[j] = self.weights[j] - self.ita*self.dweights[j]
            print("Final Error : ",self.error(y))
            k = k + 1000
            print(k/1000)
        
        
    def predict(self,X_test):
        self.forward_propagation(X_test)
        predicted = np.argmax(self.Y_hat,axis = 1)
        return predicted
    
    def accuracy(self,Y):
        predicted = np.argmax(self.Y_hat,axis = 1)
        print(predicted)
        print("Accuracy score : ",100*(accuracy_score(predicted,np.array(Y))))
            

In [10]:
k = Neural_network()
k.initialization()
#k.forward_propagation(X_train)
k.gradient_descent(X_train,Y_train)

Final Error :  0.25216174028581223
1.0
Final Error :  0.15279660990643226
2.0
Final Error :  0.12406790352870053
3.0
Final Error :  0.109084198733904
4.0
Final Error :  0.11270274023754989
5.0
Final Error :  0.09503860033259413
6.0
Final Error :  0.10398220617496526
7.0
Final Error :  0.08636651888753302
8.0
Final Error :  0.07544826331296244
9.0
Final Error :  0.07819652055789195
10.0
Final Error :  0.07823820025358823
11.0
Final Error :  0.09147445809636964
12.0
Final Error :  0.0760546690013474
13.0
Final Error :  0.07746433973545193
14.0
Final Error :  0.07602985549540336
15.0
Final Error :  0.10682326060917907
16.0
Final Error :  0.0881900103103215
17.0
Final Error :  0.11601037430800788
18.0
Final Error :  0.10192704344612195
19.0
Final Error :  0.1033232068402081
20.0
Final Error :  0.09585719842086636
21.0
Final Error :  0.1051999990285222
22.0
Final Error :  0.11325243720670254
23.0
Final Error :  0.10348157854801622
24.0
Final Error :  0.11442798308738293
25.0
Final Error :  

In [11]:
# X_test = X_test[0:1000]
# Y_test = Y_test[0:1000]

k.predict(X_test)
k.accuracy(Y_test)

[1 3 9 ... 7 9 7]
Accuracy score :  83.525


In [12]:
X_test = pd.read_csv('apparel-test.csv')
X_test = (X_test - np.mean(X_test))/np.std(X_test)
ans = k.predict(X_test)
ans_df = pd.DataFrame(ans)
ans_df.to_csv('predict.csv',index=False)